# Simple Token Distribution

A cadCAD sketch exploring an inflationary token model managed and distributed by community contributors.


## Check cadCAD

This cell doesn't do anything, but it does ensure that you know what version of cadCAD you're running. That way if/when you encounter technical difficulties you can tell the community which version of cadCAD you're running. Might save you hours of pain if all you need to do is upgrade to the latest version. 

In [27]:
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /usr/local/lib/python3.8/site-packages
Requires: pandas, funcy, fn, pathos
Required-by: cadCAD-diagram
Note: you may need to restart the kernel to use updated packages.


## Import Stuff

These are the libraries you'll need (cadCAD stuff) and that could be useful (python stuff) for your cadCAD model.

In [28]:
# Python
import numpy as np
import random as random

# cadCAD
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# Notebook setup
experiment = Experiment()
del configs[:]

## Initial Params

These are the parameters that control the behavior of a system. For example, if you were modeling a Web3 protocol these would be the parameters within a system that token holders could vote to modify. All of the parameters of a protocol that can be modified should be accessible here. That way it's easy to modify and run similations to explore how parameter changes might affect the network.

In [29]:
# The initial states of the cadCAD model
genesis_states = {
    # A variable that tracks the total amount of tokens in the system
    'tokens': 1,
    # Contributor stats
    'contributors': {
        i: {'balance': 1, 'skill_level': 5, 'impact': 0} for i in range(1)    
    },
    # The pool of tokens to reward contributors
    'treasury': 0,
}

# The parameters to run the model
params = {
    # Timesteps
    'T': range(52), 
    # Parallel runs
    'N': 1,
    # Params passed to policy and state update functions
    'M': {
        # Number of tokens to mint to the treasury 
        'weekly_token_minting': [1000],
        # Number of new contributors
        'weekly_new_contributors': [3],
        # Maximum amount of effort per week
        'maximum_effort': [7],
        # Maximum skill_level of any contributor
        'maximum_skill_level': [10]
    }
}

## Policy Functions

Policy functions are like flows in stock and flow diagrams. They modify the inputs to state update functions.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks. 

In [30]:
# Mint tokens on a weekly basis (each timestep)
def p_mint_tokens(params, step, sH, s):
    minted_tokens = params['weekly_token_minting']
    return ({'minted_tokens': minted_tokens})

# Do work to earn tokens
def p_effort(params, step, sH, s):
    effort = random.randint(1, params['maximum_effort'])
    return ({'effort': effort})

# New community contributors
def p_new_community_contributors(params, step, sH, s):
    latest_contributor_key = list(s['contributors'])[-1]
    new_contributors = {
        i: {
            'balance': 0,
            'skill_level': random.randint(1, params['maximum_skill_level'])
        } for i in range(latest_contributor_key + 1, latest_contributor_key + params['weekly_new_contributors'])
    }
    return ({'new_contributors': new_contributors})


## State Update Functions

These functions take in inputs (state variables and policies) and modify the state.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks.

In [31]:
# Update token count based on weekly minting
def s_update_token_count(params, step, sH, s, _input):
    y = 'tokens'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Put newly minted tokens into the community treasury
def s_mint_to_treasury(params, step, sH, s, _input):
    y = 'treasury'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Give new contributors accounts
def s_new_contributor_accounts(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    x.update(_input['new_contributors'])
    return (y, x)

# Update community treasury based on contribution rewards
def s_distribute_treasury(params, step, sH, s, _input):
    y = 'treasury'
    x = s[y]
    for i in s['contributors']:
        impact = s['contributors'][i]['skill_level'] * _input['effort']
        if x >= impact:
            x -= impact
        else:
            break
    return (y, x)

# Distribute rewards to contributors
def s_reward_contributions(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    for i in x:
        impact = s['contributors'][i]['skill_level'] * _input['effort']
        if s['treasury'] >= impact:
            x[i]['balance'] += impact
        else:
            break
    return (y, x)


## State Update Block

If you're approaching cadCAD from Web3 you can think of the state of the model as something that evolves in blocks. Each block has a set of actions that updates the state. Those transactions then get batched into blocks to be processed together. In cadCAD blocks are called "`partial_state_update_blocks`." As you can see below, this is an array that is very similar to a "block" in a blockchain in that it represents a set of actions to update the state. That state is then updated across many timesteps. This is similar to how the state of a blockchain is updated over many timesteps as new blocks are added.



In [32]:
partial_state_update_blocks = [
    { 
        # Mint tokens
        'policies': { 
            'minted_tokens': p_mint_tokens
        },
        # Update token count and community treasury
        'variables': {
            'tokens': s_update_token_count,
            'treasury': s_mint_to_treasury
        }
    },
    {
        # Add new contributors
        'policies': {
            'new_contributors': p_new_community_contributors
        },
        # Give contributors accounts
        'variables': {
            'contributors': s_new_contributor_accounts
        }
    },
    {
        # Do work
        'policies': {
            'effort': p_effort
        },
        # Recognize and reward contributors
        'variables': {
            'treasury': s_distribute_treasury,
            'contributors': s_reward_contributions
        }
    }
    # TO DO: 
    # Add in intermediate step between doing work and getting rewarded
    # At this step contributors would earn impact points
    # Then in the reward stage the weekly rewards would be distributed between contributors based on impact points
    # 20% of rewards would stay in the treasury for long-term projects, and 80% would go to weekly contributors 
]

## Running the cadCAD Engine

In [33]:
experiment.append_configs(
    initial_state = genesis_states,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = config_sim(params)
)

exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs) 
raw_system_events, tensor_field, sessions = simulation.execute() 



                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (52, 4, 1, 3)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.10s


## Data Visualization

This is often half the battle. Not only do need to design and build a cadCAD model, but you need to understand how it's working and be able to effectively communicate that to other people. A picture says a thousand words, thus enter data viz. Getting good at using python data viz libraries is probably the highest leverage thing you can do after you learn the cadCAD basics.

In [34]:
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

tokens  \
subset run timestep substep           
0      1   0        0             1   
           1        1          1001   
                    2          1001   
                    3          1001   
           2        1          2001   
...                             ...   
           51       2         51001   
                    3         51001   
           52       1         52001   
                    2         52001   
                    3         52001   

                                                                  contributors  \
subset run timestep substep                                                      
0      1   0        0        {0: {'balance': 1, 'skill_level': 5, 'impact':...   
           1        1        {0: {'balance': 1, 'skill_level': 5, 'impact':...   
                    2        {0: {'balance': 1, 'skill_level': 5, 'impact':...   
                    3        {0: {'balance': 6, 'skill_level': 5, 'impact':...   
           2        1        {0: {'balance': 6, 'skill_level': 5, 'impact':...   
...                                                                        ...   
           51       2        {0: {'balance': 1066, 'skill_level': 5, 'impac...   
                    3        {0: {'balance': 1086, 'skill_level': 5, 'impac...   
           52       1        {0: {'balance': 1086, 'skill_level': 5, 'impac...   
                    2        {0: {'balance': 1086, 'skill_level': 5, 'impac...   
                    3        {0: {'balance': 1096, 'skill_level': 5, 'impac...   

                             treasury  simulation  
subset run timestep substep                        
0      1   0        0               0           0  
           1        1            1000           0  
                    2            1000           0  
                    3             983           0  
           2        1            1983           0  
...                               ...         ...  
           51       2            1035           0  
                    3               3           0  
           52       1            1003           0  
                    2            1003           0  
                    3               7           0  

[157 rows x 4 columns]

In [35]:
raw_system_events

,
   71: {'balance': 532, 'skill_level': 7},
   72: {'balance': 228, 'skill_level': 3},
   73: {'balance': 657, 'skill_level': 9},
   74: {'balance': 511, 'skill_level': 7},
   75: {'balance': 330, 'skill_level': 5},
   76: {'balance': 528, 'skill_level': 8},
   77: {'balance': 512, 'skill_level': 8},
   78: {'balance': 256, 'skill_level': 4},
   79: {'balance': 57, 'skill_level': 1},
   80: {'balance': 57, 'skill_level': 1},
   81: {'balance': 106, 'skill_level': 2},
   82: {'balance': 424, 'skill_level': 8},
   83: {'balance': 376, 'skill_level': 8},
   84: {'balance': 94, 'skill_level': 2},
   85: {'balance': 120, 'skill_level': 3},
   86: {'balance': 280, 'skill_level': 7},
   87: {'balance': 108, 'skill_level': 3},
   88: {'balance': 324, 'skill_level': 9},
   89: {'balance': 31, 'skill_level': 1},
   90: {'balance': 310, 'skill_level': 10},
   91: {'balance': 216, 'skill_level': 9},
   92: {'balance': 168, 'skill_level': 7},
   93: {'balance': 220, 'skill_level': 10},
   94: {'ba